In [1]:
from captum.attr import ShapleyValueSampling
from tqdm import trange

from load_data import load_data
from train_models import *
from segmentation import *
from utils import *
import os

In [8]:
# to utils.py

def change_points_to_lengths(change_points, array_length):
    # change points is 1D iterable of idxs
    # assumes that each change point is the start of a new segment, aka change_points = start points
    start_points = np.array(change_points)
    end_points = np.append(change_points[1:], [array_length])
    print(start_points, end_points)
    lengths = end_points - start_points
    return lengths

def lengths_to_weights(lengths):
    # lengths is 1D iterable of positive ints
    start_idx = 0
    end_idx = 0
    segment_weights = 1 / lengths
    weights = np.ones(lengths.sum())
    for segment_weight, length in zip(segment_weights, lengths):
        end_idx += length
        weights[start_idx: end_idx] = segment_weight
        start_idx = end_idx
    return weights


In [9]:
# device for torch
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest']
}

In [10]:
# load data
dataset_name = 'gunpoint'
X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
n_samples, n_chs, ts_length = X_test.shape

# train model
predictor_name = 'randomForest'
if predictor_name=='resNet':
	clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
elif predictor_name=='miniRocket':
	clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
elif predictor_name=="randomForest":
	clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
else:
	raise ValueError(
		"predictor not found"
	)

# initialize data structure meant to contain the segments
# TODO can I be cleaner here?
segments =  np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (
	np.empty( X_test.shape[0] , dtype=object))

# create a dictionary to be dumped containing attribution and metadata
results = {
	'attributions' : {},
	'segments' : segments,
	'y_test_true' : y_test,
	'y_test_pred' : preds,
	'label_mapping' : enc,
}

training random forest
random forest accuracy is 0.9133333333333333


In [13]:
# define different background to be used and number of samples as n_background
# TODO set a number of TOTAL sampling regardless of the background type?
n_background = 50
background_types = ["average", "zero","sampling"] # zero, constant, average, multisample
for bt in background_types:
	results['attributions'][bt] = np.zeros( X_test.shape ,dtype=np.float32 )

In [12]:
with torch.no_grad():
	
	SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] \ 
		else ShapleyValueSampling(forward_classification)
	
	for i in trange ( n_samples ) : #
		
		# get current sample and label
		ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )

		# get segment and its tensor representation
		current_segments = get_claSP_segmentation(ts)[:X_test.shape[1]]
		results['segments'][i] = current_segments
		mask = get_feature_mask(current_segments,ts.shape[-1])

		ts = torch.tensor(ts).repeat(1,1,1)	#TODO use something similar to np.expand_dim?

		for background_type in background_types:

			# background data
			if background_type=="zero":
				background_dataset = torch.zeros((1,) + X_train.shape[1:])
			elif background_type=="sampling":
				background_dataset = sample_background(X_train, n_background)
			elif background_type=="average":
				background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

			# for sampling strategy repeat the ts many times as the background dataset size
			ts = ts.repeat(background_dataset.shape[0],1,1) if background_type=="sampling" else ts

			# different call depending on predictor type
			if predictor_name in predictors['scikit']:
				# if using random forest flat everything
				if predictor_name=="randomForest":
					ts = ts.reshape( -1, n_chs*ts_length); mask = mask.reshape( -1, n_chs*ts_length);
					background_dataset = background_dataset.reshape( -1, n_chs*ts_length)
				
				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
			
			elif predictor_name in predictors['torch']:
				# if use torch make sure everything is on selected device
				ts = ts.to(device); y = y.to(device)
				mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
			
			# 'un-flatten' for randomForest
			if predictor_name=="randomForest":
				tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

			# store current explanation in the data structure; if sampling store the mean
			results['attributions'][background_type][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
				background_type=="sampling" else tmp[0].cpu().numpy()

100%|██████████| 150/150 [18:10<00:00,  7.27s/it]


In [14]:
 # normalized weights
weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results["segments"])))
results["attributions"][background_type] *= weights

[  0   7  16  47  96 109 118] [  7  16  47  96 109 118 150]
[  0  35  45  68  78 101 132] [ 35  45  68  78 101 132 150]
[  0  23  47  69 106 119 131] [ 23  47  69 106 119 131 150]
[  0  27  61  96 111 119 135] [ 27  61  96 111 119 135 150]
[0] [150]
[  0   9  19  42  63 112 129] [  9  19  42  63 112 129 150]
[ 0 56 64 72 86 99] [ 56  64  72  86  99 150]
[  0 123] [123 150]
[  0  27  42  80 110 124 141] [ 27  42  80 110 124 141 150]
[  0  15  41  57  87 100 132] [ 15  41  57  87 100 132 150]
[  0  80  87  99 108 119 137] [ 80  87  99 108 119 137 150]
[ 0 10 39 46 63 88 97] [ 10  39  46  63  88  97 150]
[  0   9  18  39  77 114 139] [  9  18  39  77 114 139 150]
[  0  79  99 131 138] [ 79  99 131 138 150]
[  0  15  65  88 103 119 138] [ 15  65  88 103 119 138 150]
[ 0 12 83] [ 12  83 150]
[  0  34  69 120] [ 34  69 120 150]
[ 0  8 19 34 42 77 98] [  8  19  34  42  77  98 150]
[0] [150]
[  0  33  40  48  65  77 106] [ 33  40  48  65  77 106 150]
[  0  26 129] [ 26 129 150]
[  0  16  38  7

In [16]:
# dump result to disk
file_name = "_".join ( ( predictor_name, dataset_name ) )+".npy"
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

In [10]:
results['attributions'][background_type].sum(axis=(1,2))

array([ 1.70120507e-01,  1.52841201e-02,  9.28617418e-02,  1.13407120e-01,
        1.83220327e-01,  2.10560542e-02,  1.27464145e-01,  1.06733963e-01,
        1.75181329e-02,  1.57249138e-01,  1.40664607e-01,  1.49215162e-01,
        1.59901679e-02,  1.27517417e-01,  1.47278994e-01,  1.60658777e-01,
        2.17270672e-01,  5.09824380e-02,  3.32470946e-02,  3.66214737e-02,
        1.30641073e-01,  1.23184487e-01,  1.96215227e-01,  1.38463348e-01,
        8.75410587e-02,  9.70023274e-02,  1.71967596e-01,  1.50519937e-01,
        5.04426882e-02,  1.61408171e-01,  2.16436416e-01,  1.41229391e-01,
        1.41947210e-01,  1.62317842e-01,  1.64650887e-01,  1.64535791e-01,
        1.46388069e-01,  1.16750002e-01,  1.22873716e-01,  4.31026220e-02,
        1.57310590e-01,  1.00012690e-01,  1.55799270e-01,  1.55326679e-01,
        8.71996731e-02,  1.57125652e-01,  1.36757612e-01,  3.19495164e-02,
        1.55807734e-01,  3.47837806e-02,  9.90368426e-05,  1.06972188e-01,
        1.50174171e-01,  